In [50]:
import numpy as np
import pandas as pd
import os
import multiprocessing as mul
from multiprocessing import Process

## IMPORTING AND SORTING THE DATA

WE HAVE CHOSEN ICECUBE 2008-18 DATA FOR THIS STUDY


THE ICE CUBE DATA SET HAS 1134450 NEUTRINO EVENTS


We select neutrino events with Energy >= 100TeV = 10^5 GeV

i.e log10(E/GeV) > 5


There are 192107 such neutrino events in this data

The ms pulsars are taken from the ATNF Catalogue

There are 441 pulsars (as of May 2022 when the study started)

All the pulsars lie in the declination range of -87 to +87 degrees

In [2]:
####
#### IMPORTING AND SPLITTING ICDATA $$$


path = "/media/darkwake/VIB2/Project-IceCube/icecube_10year_ps/events"
filenames = ["IC40_exp.csv", "IC59_exp.csv","IC79_exp.csv", "IC86_I_exp.csv", "IC86_II_exp.csv",
"IC86_III_exp.csv", "IC86_IV_exp.csv", "IC86_V_exp.csv", "IC86_VI_exp.csv", "IC86_VII_exp.csv"]
file = filenames[0]
f = open(os.path.join(path, file), 'r')
lines = f.readlines()
column=lines[0].split()
column.pop(0)
content = []
for file in filenames:
    f = open(os.path.join(path, file), 'r')
    lines = f.readlines()
    for line in lines[1:]:
        content.append(line.split())
    f.close()
icdata = pd.DataFrame(content, columns=column)
icdata['log10(E/GeV)'] = [float(i) for i in icdata['log10(E/GeV)']]

icdata = icdata.sort_values('log10(E/GeV)')
icdata = icdata.reset_index()
icdata = icdata.drop('index', axis=1)
icdata2 = icdata[icdata['log10(E/GeV)'] > 5]
icdata2 = icdata2.reset_index()
icdata2 = icdata2.drop('index', axis=1)
icdata2

#IMPORTING MSPDATA
f = open("/media/darkwake/VIB2/Project-IceCube/10milsecpsr.txt", 'r')
lines = f.readlines()

content=[]
column=lines[3].split()
for line in lines[:]:
    content.append(line.split())
    #the INITAL DATABASE IS CLUTTERED SO WE REMOVE THE NULL COLUMNS AND OTHER CLUTTER
mspdata = pd.DataFrame(content).drop(range(0,6)).dropna().drop([2,6,8,10,11,13,14], axis=1)
f.close()
line = []
lines = []

mspdata.columns = column
column = []
content=[]
mspdata = mspdata.sort_values('DECJD')
mspdata.dropna(inplace=True)
mspdata = mspdata.reset_index()
mspdata = mspdata.drop(["index", "#"], axis=1)

In [3]:
icdata = []
icdata2

,MJD[days],log10(E/GeV),AngErr[deg],RA[deg],Dec[deg],Azimuth[deg],Zenith[deg]
0,58192.76586876,5.01,0.20,304.076,-27.773,234.673,62.284
1,57692.17772634,5.01,0.30,328.288,-24.485,225.360,65.593
2,56499.25268749,5.01,0.20,215.304,-72.162,269.459,17.774
3,55742.58579947,5.01,0.29,121.980,-37.347,97.098,52.613
4,57168.08498961,5.01,0.65,209.099,-81.386,154.309,8.532
...,...,...,...,...,...,...,...
192102,57850.41489986,7.00,1.48,299.091,-19.242,135.893,70.801
192103,56616.28975047,7.01,1.24,300.952,-48.575,312.569,41.469
192104,57741.31320782,7.04,3.27,226.680,-7.559,64.185,82.379
192105,58099.46879008,7.26,3.32,317.013,-34.929,22.825,55.146


In [4]:
mspdata

,NAME,Gl,Gb,RAJD,DECJD,P0,F0,DIST
0,J1853-0008g,33.014,-0.453,283.30,-0.13,0.00282,354.609929,4.554
1,J1625-0021,13.890,31.827,246.2931579,-0.358044,0.0028336138772234,352.90623328676,0.951
2,J1852-0044g,32.395,-0.560,283.11,-0.73,0.00241,414.937759,4.492
3,J1835-0114,29.990,3.007,278.841325,-1.242669,0.005116387644239,195.4503977286,3.452
4,J1901-0125,32.817,-2.902,285.39083,-1.42472,0.00279,358.422939,2.360
...,...,...,...,...,...,...,...,...
436,J1944+0907,47.160,-7.357,296.03887491,9.12306353,0.005185201908798642,192.8565208431179,1.218
437,J0023+0923,111.383,-52.849,5.82032291,9.38996121,0.003050203104754390,327.8470205611185,1.818
438,B1855+09,42.290,3.060,284.401626217,9.721442158,0.005362100549682627,186.4940783438289,1.200
439,J2234+0944,76.280,-40.438,338.69522573,9.74172845,0.003627027895734199,275.7078326240928,1.587


In [5]:
max(mspdata['P0'])

'0.0098613486703673'

In [6]:
def hvovec(lon1, lat1, lon2, lat2):

    #Convert decimal degrees to Radians:
    lon1 = np.deg2rad(lon1)
    lat1 = np.deg2rad(lat1)
    lon2 = np.deg2rad(lon2)
    lat2 = np.deg2rad(lat2)

    #Implementing Haversine Formula: 
    dlon = np.subtract(lon2, lon1)
    #dlat = np.subtract(lat2, lat1)

    a = np.add(np.multiply(np.sin(lat1), np.sin(lat2)), np.multiply(np.multiply(np.cos(lat1), np.cos(lat2)), np.cos(dlon)))

    return np.abs(np.rad2deg(np.arccos(a)))

In [7]:
msra = [float(i) for i in mspdata['RAJD']]
msdec = [float(i) for i in mspdata['DECJD']]
icra = [float(i) for i in icdata2['RA[deg]']]
icdec = [float(i) for i in icdata2['Dec[deg]']]
icang = [float(i) for i in icdata2['AngErr[deg]']]
p = len(msra)
lg = len(icra) // p + 1

In [8]:
def angfinder(b):
    ang = []
    for a in range(lg):
        
        if a != lg - 1:
        #try:
            ilo = icra[a*p:a*p + p]
            ila = icdec[a*p:a*p + p]
            lo = msra[b] * np.ones(p)
            la = msdec[b] * np.ones(p)
            temp = hvovec(ilo, ila, lo, la)
            for tt in range(len(temp)):
                if temp[tt] > 20:
                    temp[tt] = -1
            ang.extend(temp)
        else:
        #except:
            ilo = icra[a*p:]
            ila = icdec[a*p:]
            ext = len(ilo)
            lo = msra[b] * np.ones(ext)
            la = msdec[b] * np.ones(ext)
            temp = hvovec(ilo, ila, lo, la)
            #ang.extend(hvovec(ilo, ila, lo, la))
            for tt in range(len(temp)):
                if temp[tt] > 20:
                    temp[tt] = -1
            ang.extend(temp)
            
        
    return ang

pool = mul.Pool()
op_async = pool.map_async(angfinder, range(0,p))
aang = op_async.get()
op_async = []
pool = []

newdata = icdata2
#icdata = []
#icdata2 = []

In [9]:
pool = []
op_async = []

we only select those neutrino events, whose angular distance  is within $20^{\circ}$ of a MSP.  For a dataset of $N$ events, if $n_s$ signal events are attributed to a millisecond pulsar, the probability density of an individual event $i$ is given by:
\begin{equation}
P_i = \frac{n_s}{N} S_i + (1-\frac{n_s}{N}) B_i,
%\label{eq1}
\end{equation}
where $S_i$ and $B_i$ represent the signal and background pdfs, respectively.
The likelihood function ($\mathcal{L}$) of the entire dataset, obtained from the product of each individual PDF can be written as 
\begin{equation}
\mathcal{L} (n_s) = \prod_{i=1}^N P_i,
\end{equation}
where $P_i$ is the same as in eq1 above. The signal PDF is given by:
\begin{equation}
\large
S_i = \frac{1}{2\pi\sigma_i^2}e^{\frac{-(|\theta_i-\theta_s|)^2}{2\sigma_i^2}}
\end{equation}
where $|\theta_i-\theta_s|$ is the angular distance between the  MSP and the neutrino, whereas $\sigma_i$ is the angular uncertainty in the neutrino position.
The background PDF is determined from the average number of events per solid angle  within a declination of $\pm 3^{\circ}$ after averaging over RA. We do not incorporate  the energy information, since the pubic IceCube catalog only contains the energy of the reconstructed muon. The detection statistic used to ascertain the presence of a signal is given by:
\begin{equation}
TS (n_s) = 2 \log \frac{\mathcal{L} (n_s)}{\mathcal{L} (0)}
\end{equation}
 If the null hypothesis is true, $TS (n_s)$ obeys the $\chi^2$ distribution for one degree of freedom. We calculated $TS (n_s)$ for all MSPs in the  catalog.

### FOR $\nu_i$ and $p_j$ and  N = len(icdata2) = Total no.of $\nu$ samples
### ns[i] = no.of $\nu$ events with angles $<= 20 \deg$ with $psr_i$
### $S_i = \frac{1}{2\pi\sigma_i^2}e^{\frac{-(|\theta_i-\theta_s|)^2}{2\sigma_i^2}}$
### $|\theta_i-\theta_s|$ = aang[i][s]
### $\sigma_i$ = icang[i]           and                  sg =      $\sigma_i^2$
###

In [10]:
N = len(icdata2)

ns = []
for i in aang:
    count = 0
    for j in i:
        if j != -1:
            count += 1
    ns.append(count)
ns = np.array(ns)
    

In [57]:
Ns = np.sum(ns)//2

In [58]:
Ns

1665941

n2 = []
for i in aang:
    count = 0
    for j in i:
        if j != -1:
            if j < 5:
                count += 1
    n2.append(count)

for i in aang[0]:
    if i < 5 and i!= -1:
        print(aang[0].index(i))

S_ij(10754,0)

[msra[0], msdec[0]]

[icra[10754], icdec[10754], icang[10754]]

[icra[119640], icdec[119640], icang[119640]]


S_ij(119640,0)

for i in aang[0]:
    if i < 6 and i!= -1 and i >= 5:
        print(aang[0].index(i))

S_ij(31410, 0)

for i in aang[0]:
    if i < 7 and i!= -1 and i >= 6:
        print(aang[0].index(i))

np.exp(-0.05)/150

aang[0][39160] **

S_ij(39160,0)

for i in aang[0]:
    if i < 8 and i!= -1 and i >= 7:
        print(aang[0].index(i))

S_ij(99351,0)
#aang[0][142]

ang = np.deg2rad(aang[0][99351]) ** 2

    #if ang < 20 and ang != -1:
sg = np.deg2rad(icang[99351]) ** 2
np.exp(-1 * ang / (2 * sg)) / (2 * np.pi * sg)

ang = np.deg2rad(aang[0][99351]) ** 2
err = np.deg2rad(icang[99351]) **2
np.exp(-ang/(2 * err))/(2 * np.pi * err)

np.exp(-ang/(2 * err))

-ang/(2 * err)

err

a2 = aang[0][99351] ** 2
a2

icang[99351]

In [11]:

#S_ij for ith neutrino and jth pulsar
def S_ij(i):
    arr = []
    sg = icang[i] ** 2
    for j in range(0,p):
        ang = aang[j][i]
        if ang < 20 and ang != -1:
            ang = ang ** 2
            
        arr.append(np.exp(-ang / (2 * sg)) / (2 * np.pi * sg)*ns[j])
        #return
    else:
        arr.append(0)
        #return -1
    
    return np.average(arr)
def sij():
    pool = mul.Pool()
    op_async = pool.map_async(S_ij, range(len(icdec)))
    si = op_async.get()
    pool = []
    op_async = []
    return si
#for i in range(len(si)):
'''def Si():
    si = sij()
    ssi = []
    for i in range(len(si)):
        for j in range(len(si[i])):
            si[i][j] = si[i][j]/ns[i]
        ssi.append(np.average(si[i]))
    
    return ssi
'''

#S = Si()

In [ ]:
S = 

def Si():
    si = sij()
    pool = mul.Pool()
    op_async = pool.map_async(Sib, si, range(len(icdec)))
    si = op_async.get()
    pool = []
    op_async = []
    return si

ss = sij()

ssss
s

ssi = []
i=0
for i in range(len(ss)):
    for j in range(len(ss[i])):
        ss[i][j] = ss[i][j]/ns[i]
    ssi.append(np.average(ss[i]))
    

len(ssi)

2103.12813

For each sample, given the Dec δi of an event, the background PDF is determined by the relative number of
events in δi ± $3^◦$ divided by the solid angle.

so calculate total no of events within delta +/- 3
and then divide by 2 pi (sin[delta+3]-sin[delta-3])

you can choose all events
withiin delta +/- 3


In [12]:
def bg(i):
    #Calculating total no of neutrino events within delta +/- 3 of ith neutrino
    count = 0
    for j in icdec:
        if abs(i - j) <= 3:
            count+=1 
    #calculating solid angle with 3deg
    sang = 2 * np.pi * (np.sin(np.deg2rad(icdec[i] + 3)) - np.sin(np.deg2rad(icdec[i] - 3)))
    return count/sang

def Bi():
    pool = mul.Pool()
    op_async = pool.map_async(bg, range(len(icdata2)))
    return op_async.get()

B = np.array(Bi())

In [42]:
B

array([ 91.07945132,  75.18440728, 213.43889111, ...,   0.        ,
         0.        ,   0.        ])

In [14]:
sssss = 0
for i in B:
    if i == 0:
        sssss+=1
sssss

192029

In [15]:
len(B)

192107

In [16]:
def Pr(ns, N, S, B):
    nsN = ns/N
    return np.add(np.multiply(nsN , S), np.multiply(np.subtract(1, nsN), B))

In [17]:
S = np.array(sij())
l = np.prod(Pr(8, N, S, B))

In [25]:
tt = Pr(8, N, S, B)

In [36]:
counter = 0
for i in tt:
    if i < 1:
        counter +=1
counter

152634

In [38]:
counter - (len(tt) - counter)

113161

ans is in the order of $10^{113161}$

In [21]:
def Ls(S, B, N, ns1):
    return np.prod(Pr(ns1, N, S, B))

In [40]:
L_s = [Ls(S, B, N, i) for i in range(0, 441)]
L_s

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0

def L_n(S, B, N, ns):
    L = np.ones(len(ns))
    #pool = mul.Pool()
    #op_async = pool.map_async(Pr, ns, N, S, B)
    #l = op_async.get()
    l = []
    i = 0
    for i in ns:
        l.append(Pr(i, N, S, B))
    i = 0
    for i in range(0, len(l)):
        for j in range(len(l[i])):
            L[i] *= l[i][j]
    return L
L = L_n(S, B, N, range(0,5))


In [ ]:
def TS(s, S, B, N):
    ts = []
    for i in range(0,4):
        ts.append(2 * np.log10(Ls(S, B, N, i)/Ls(S, B, N, 0)))
    return ts

We use the same unbinned likelihood method as in ~\cite{Hooper,Kamionkowski,LuoZhang} which was first proposed in ~\cite{Montaruli10}. Similar to ~\cite{LuoZhang}, we only select those neutrino events, whose angular distance  is within $20^{\circ}$ of a MSP.  For a dataset of $N$ events, if $n_s$ signal events are attributed to a millisecond pulsar, the probability density of an individual event $i$ is given by:
\begin{equation}
P_i = \frac{n_s}{N} S_i + (1-\frac{n_s}{N}) B_i,
%\label{eq1}
\end{equation}
where $S_i$ and $B_i$ represent the signal and background pdfs, respectively.
The likelihood function ($\mathcal{L}$) of the entire dataset, obtained from the product of each individual PDF can be written as 
\begin{equation}
\mathcal{L} (n_s) = \prod_{i=1}^N P_i,
\end{equation}
where $P_i$ is the same as in Eq.~\ref{eq1}. The signal PDF is given by
\begin{equation}
S_i = \frac{1}{2\pi\sigma_i^2}e^{-(|\theta_i-\theta_s|)^2/2\sigma_i^2}
\end{equation}
where $|\theta_i-\theta_s|$ is the angular distance between the  MSP and the neutrino, whereas $\sigma_i$ is the angular uncertainty in the neutrino position.
The background PDF is determined from the average number of events per solid angle  within a declination of $\pm 3^{\circ}$ after averaging over RA. We do not incorporate  the energy information, since the pubic IceCube catalog only contains the energy of the reconstructed muon. The detection statistic used to ascertain the presence of a signal is given by:
\begin{equation}
TS (n_s) = 2 \log \frac{\mathcal{L} (n_s)}{\mathcal{L} (0)}
\end{equation}
 If the null hypothesis is true, $TS (n_s)$ obeys the $\chi^2$ distribution for one degree of freedom. We calculated $TS (n_s)$ for all MSPs in the  catalog.

def b_ivec(lon1, lat1, lon2, lat2):
  
    #Convert decimal degrees to Radians:
    lon1 = np.radians(lon1)
    lat1 = np.radians(lat1)
    lon2 = np.radians(lon2)
    lat2 = np.radians(lat2)

    #Implementing Haversine Formula: 
    dlon = np.subtract(lon2, lon1)
    #dlat = np.subtract(lat2, lat1)


    a = np.add(np.multiply(np.sin(lat1), np.sin(lat2)), np.multiply(np.multiply(np.cos(lat1), np.cos(lat2)), np.cos(dlon)))

    return (np.rad2deg(np.arccos(a)), a)

evts_p_srad = []
l_ic = len(icdata2)
for i in range(len(mspdata)):
#returns $B_i$ as defined above
#def B_i(i):

    r = []
    psra = float(mspdata['RAJD'][i])
    psdec = float(mspdata['DECJD'][i])
    pssa = np.cos(np.cos(np.deg2rad(psdec) + np.cos(np.deg2rad(3))) - np.cos(np.deg2rad(psdec) - np.cos(np.deg2rad(3))))
    for j in range(l_ic):
        if icdec[j] > np.deg2rad(psdec) - pssa and icdec[j] < np.deg2rad(psdec) + pssa:
            r.append(j)
    evts_p_srad.append([i,r])

    